# Introduction

Schistosomiasis is a tropical disease caused by human infective parasites of the genus Schistosoma. The larval stage, miracidia, develop in the snail intermediate host into cercariae that are infective to humans. Once a human is infected, the parasites will travel to the mesenteric vasculature connecting the liver and intestine. Here, the worms mature and lay eggs. Some eggs travel to the intestine and are excreted into an aquatic environment before hatching to miracidia. However, other eggs travel to the liver, and these eggs do not get passed on and will not continue the cycle of infection. Liver eggs are typically used in rodent hosts for experimental research use. 


There are distinct functional differences in the eggs from the distinct tissues [1]. However, there is no understanding of the differences in miracidia from each tissue’s eggs. We utilized bioinformatics techniques and performed RNA-seq analysis on miracidia from infected mouse livers and intestines. We sought to understand transcriptomic differences in these miracidia and potentially find specialized functions dependent on location of maturation.

# Methods

### RNA Extraction, Library Preparation, and Sequencing

(Going to be written by Dr. Wheeler)

### Read QC and Alignment

To perform QC (quality control) on our sequenced reads, we first performed FASTQC (v. 0.12.1) analysis to compile and score the reads overall. To view these quality scores, multiqc (v. 1.17) was used to view a html file. This allowed us to be given quality scores on the reads for categories: Per base sequence quality, Per tile sequence quality, Per sequence quality score, Per base sequence content, Per sequence GC content, Per base N content, Sequence length distribution, Sequence duplication levels, Overrepresented sequences, and Adapter content. A score was associated for each of the 16 paired intestine and liver miracidia samples. After first evaluation of the scores, we found that there were areas where trimming was necessary to improve the quality of the reads. We limited the reads to be a minimum of 20 base pairs in length or more. We also trimmed the adapter sequence that was found in both read 1 (AGATCGGAAGAGCACACGTCTGAACTCCAGTCA) and read 2 (AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT) of all 16 samples. After this trim, we performed our FASTQC analysis again and found the quality scores to have improved from the first run-through. We used these trimmed reads for our next step; alignment.

To perform alignment of our reads to the reference genome, we utilized STAR (v. 2.7.10a) to perform a two-pass alignment. We set certain parameters for our base alignment:
- !STAR \
--runThreadN 16 \
--runMode genomeGenerate \
--genomeDir ../Sm_Mira_IvT/genome/star \
--genomeFastaFiles ../Sm_Mira_IvT/genome/genome.fa \
--sjdbGTFfile ../Sm_Mira_IvT/genome/annotations.gtf \
--sjdbOverhang 150 \
--genomeSAindexNbases 13

After this aligned, we took those reads to perform our first pass test, with parameters as:
- !STAR \
--runThreadN 16 \
--runMode alignReads \
--genomeDir ../Sm_Mira_IvT/genome/star \
--readFilesManifest manifest_mira.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outFileNamePrefix alignment_mira/star/first_pass/

We took the output of this first pass and performed our second pass test with parameters as: 
- !STAR \
--runThreadN 16 \
--runMode alignReads \
--genomeDir ../Sm_Mira_IvT/genome/star \
--readFilesManifest manifest_mira.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outSAMattributes NH HI AS nM RG \
--outFileNamePrefix alignment_mira/star/second_pass/ \
--sjdbFileChrStartEnd alignment_mira/star/SJ.out.tab

The output from our alignment passes gave us a .bam file that we used for alignment QC analysis. To do this, Samtools (v. 1.6), Picard (v. 3.2.0), and Qualimap (v. 2.3) were used. Samtools took the output .bam files from STAR and performed analysis to output a star_stats.txt file that we could use to view summaries associated with the entirety of our reads. We used Picard and Qualimap for our next step of analysis with parameters set as:

- !picard AddOrReplaceReadGroups I=alignment_mira/star/second_pass/Aligned.sortedByCoord.out.bam O=qualimap/output.bam RGLB=temp RGPL=Illumina RGPU=1 RGSM=20 VALIDATION_STRINGENCY=LENIENT

- !qualimap bamqc -nt 32 -outdir qualimap/star/bam -bam alignment_mira/star/second_pass/Aligned.sortedByCoord.out.bam --feature-file genome/annotations.gtf
- !qualimap rnaseq -outdir qualimap/star/rnaseq -bam alignment_mira/star/second_pass/Aligned.sortedByCoord.out.bam -gtf genome/annotations.gtf

After receiving the output files and viewing our analysis, we were able to step further and begin generating counts from our aligned reads. Using Picard tools once again, parameters were set as: 
- !picard MarkDuplicates I=alignment_mira/star/second_pass/Aligned.sortedByCoord.out.bam M=counting_mira/logs/star_duplicates O=counting_mira/dedup/star.bam VALIDATION_STRINGENCY=SILENT

This allowed us to locate duplicated reads and ensure that overrepresentation wasn't going to be influenced by these multiple mapped alignments. We further used FeatureCounts (v. 2.0.6) to get our counts from our .bam files. We set parameters as: 

- !featureCounts -T 32 \
    counting_mira/dedup/star.bam \
    -T 32 \
    -p \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a ../Sm_Mira_IvT/genome/annotations.gtf \
    -o counting_mira/star_counts.tsv \
    --verbose

We ran a multiqc report on the .tsv files output and were able to view now the entirety of our analysis and the quality of each step. This provided useful insight to our aligned reads and how well we had performed the steps. This set our foundation for identification of differentially expressed genes (DEGs) and the gene ontology (GO) behavior of these genes.

### Identification of differentially expressed genes and GO analysis

With our reads analyzed and passed through our set parameters, now we could perform analysis to find our DEGs. We utilized R (v. 4.2.1) to perform our differential expression analysis. The following packages were used: Tidyverse, DESeq2, and broom. First, we converted our counts processed through STAR into a dataframe. We altered our dataframe to remove excess lines of writing and to remove genes with small values for total counts from the dataframe. We filtered the dataframe and turned it into a matrix of genes that contained our total counts of reads that aligned. Using this matrix, we performed a PCA analysis on the distribution of our STAR count matrix. 

For the next analysis of our differentially expressed genes, we performed a volcano plot analysis in R as well. Taking the same count matrix from STAR reads, we altered some of the variables associated with tissue type and replicate number. This made our metadata easier to work with for our analysis. We used the DESeq2 package to analyze our reads from the matrix and from our metadata with tissue being one of our variables for design. This was stored and used to create our volcano plot with our x-axis being log2FoldChange and our y-axis being -log10(p-adjusted). 

After we found that our reads expressed DEGs, we performed further analysis into these genes to understand any biological or functional importances associated with them in the S. mansoni species. We once again used R for this analysis, with packages: Tidyverse, and httpgd. We imported our raw counts, our normalized counts (after performing DESeq2), and our DEGs results before performing our analysis. We took the normalized counts and pivoted our data to be a longer table with each gene being a separate observation. We took our DEGs table and filtered to have a p-adjusted value of less than or equal to 0.05, as well as a log2FoldChange greater than or equal to 1 or less than or equal to -1. This gave us a table that contained our DEGs with specific values for our p-adjusted values and log2FoldChange, signifiying genes that are significant enough for our analysis. 

Out of the 87 genes that were significant DEGs, we chose four with two being highly expressed in the liver (log2FoldChange >= 1) and the other two being highly expressed in the intestines (log2FoldChange <= 1). We plotted these using ggplot2 in boxplot formation, allowing us to visualize the differences in expression between all four genes compared to both their liver and intestine expression values. 

To discover if any of the DEGs were biologically significant in the species, we utilized gprofiler2 to describe the gene ontology (GO) of these genes. We specifically filtered our total DEGs table to have a p-adjusted value less than or equal to 0.05, and the log2FoldChange greater than 1. Using the function gost, we input our table of gene id names and cross-referenced it to the genes in the entire database of the S. mansoni species. We identified genes related to nucleoside metabolic processes and glycosyl compound metabolic processes were present. Further analysis into other DEGs may provide other biologically significant functions that could give insight into S. mansoni behaviors and movements.

The full R scripts for this section are available on the GitHub repository: https://github.com/swill04/Schistosoma-RNA-seq

# Results

#### RNA sequencing, alignment, and sample clustering
After aligning our sequenced reads to the reference genome, we sought to confirm that the reads we obtained were clustering based upon the tissues that they were harvested from. To do this, we performed our PCA analysis in R. Below is the plot produced from this analysis.

<img src='/data/users/willetse0745/Schistosoma-RNA-seq/plots/post_deseq2.png' width='600' style="background-color:white;padding:20px;" >

In the plot, we see a clustering linked to tissue source. There are a few outliers, yet our data is clustering as expected. This result shows that miracidia from liver and intestine egg samples are transcriptomically distinct. This is an important first step prior to our other analysis as we now know that the differences we see in the miracidia samples are going to be different depending on which tissue they came from.

#### Identification of differentially expressed genes
Since we've shown that the miracidia are transcriptomically distinct from the different tissues, we next were interested in which genes between them are distinct. Another way to describe these genes are differentially expressed genes (DEGs). We performed a volcano plot analysis to visualize the spread of our genes from the samples. We used selective conditions to determine biological significance. Genes are plotted with statistical significance (p-adj <= 0.05 as –log10(p-adj)) related to the magnitude of fold change (based on log2 > 1 and < -1) outlined by the black lines. Pink points represent differentially expressed genes. Genes located on the right side are more highly expressed in the intestine miracidia and genes on the left side are more highly expressed in the liver miracidia. The plot is shown below.

<img src='/data/users/willetse0745/Schistosoma-RNA-seq/plots/volcano.png' width='600' style="background-color:white;padding:20px;" >

There are 87 differentially expressed genes in the graph. Discerning these is valuable for understanding specific genome differences between the two miracidia samples.

After finding the DEGs in our volcano plot, we next wanted to look deeper into some of the significant genes to understand their biological functions within the miracidia. I chose four genes from the volcano plot - two DEGs per tissue type - and plotted them in R using a simple boxplot. The plot is shown below.

<img src='/data/users/willetse0745/Schistosoma-RNA-seq/plots/DEGs_plot.png' width='600' style="background-color:white;padding:20px;" >

For the DEGs more highly expressed in the intestine miracidia, I chose Smp_319430 and Smp_319450. Based upon sequenced genome assemblies, I searched for the molecular function and biological processes that are known to each gene [3]. Based upon the search for Smp_319430 and Smp_319450, both of these genes are used in protein coding. There was no known functionality associated with them.

For the DEGs more highly expressed in the liver miracidia, I chose Smp_133770 and Smp_210300. Based upon the search performed using the same technique [3] for the intestinal miracidia, the genes both contained specific functions. Smp_133770 is used in breaking down nitrogen-containing compounds as well as glutamate-ammonia ligase activity. Smp_210300 is used in producing proteins to bind to metal ions to prevent overaccumulation. These genes are intriguing as they play vital roles in the survival of the worm and potentially removing them may result in the death of the worms.

#### Identification of pathways and/or GO annotations that are differentially expressed
Utilizing the gprofiler2 package available in R, our next analysis was to look at the overall DEGs and determine what the gene ontology of the genes was. We used the conditions mentioned in our above methods. This resulted in 25 genes out of the 87 being searched for in the gprofiler query. The plot is shown below.

<img src='/data/users/willetse0745/Schistosoma-RNA-seq/plots/gProfiler_smansoni_2025-05-06_20-49-34.png' width='600' style="background-color:white;padding:20px;" >

The plot describes overrepresented terms related to gene ontology and biological functions from the filtered list of selected genes. Functions of nucleoside metabolic process and glycosyl compound metabolic process are highlighted in our selected genes.

# Discussion
Based upon our RNA-seq alignment, QC, and differential expression analysis, we've discovered significant results between the transcriptomes of liver hatched miracidia versus intestine hatched miracidia. We conclude that miracidia hatched from eggs harvested from mouse livers and intestines are transcriptomically different based upon our clustering results (Fig. 3). We've determined that the reads from our distinct samples are different and contain different transcriptomes. Also, in the separate tissue samples, differentially expressed genes were found in the miracidia (Fig. 4-5). This finding aids in learning further of potential behavioral differences that may be involved in the different miracidia (explored further in future directions). Finally, biologically significant functions are differentially expressed (GO:0009116, GO:1901657 - Fig. 6). We've found that there are genes that contain biologically important functions within our DEGs.


#### Future Directions
There are a few potential experiments that could be performed next to advance this study. Testing the separate miracidia for differences in behavior and infection
capacity would allow us to see if the miracidia from the separate tissue samples have specific behaviors that influence their mechanisms of finding a snail intermediate host. Since we know that they are different transcriptomically, learning if the miracidia have different abilities or functions they perform when searching for a snail would be useful for preventing infection. Also, utilizing the DEGs found in this study for gene “knockout” experiments to understand any functional dependencies that may be occurring would allow us to develop preliminary methods for drug treatments or anthelmintic therapies that could aid in treatment of schistosomiasis. Finally, exploring other DEGs found for other important biological functions could provide useful information into the rest of the genome of the worms and further the methods of interaction they use in their varying environments.

# References

1.Kristýna Peterková, Lukáš Konečný, Tomáš Macháček, Jedličková L, Winkelmann F, Sombetzki M, et al. Winners vs. losers: Schistosoma mansoni intestinal and liver eggs exhibit striking differences in gene expression and immunogenicity. PLoS Pathogens. 2024 May 30;20(5):e1012268–8.

2.Hudak D, Johnson D, Chalker A, Nicklas J, Franz E, Dockendorf T, et al. Open OnDemand: A web-based client portal for HPC centers. Journal of Open Source Software. 2018 May 14;3(25):622.
‌

3.Howe KL, Bolt BJ, Shafie M, Kersey P, Berriman M. WormBase ParaSite − a comprehensive resource for helminth genomics. Molecular and Biochemical Parasitology. 2017 Jul;215:2–10.